# Tests
Various tests performed on the files to explore the results coming from the model etc.

In [26]:
import pandas as pd
import os
import csv
from src.utils import *
import IPython.display as ipd
from src.audio_utils import read_audio
import librosa
from attacks.attacks_utils import *
from attacks.sp_utils import retrieve_single_audio

In [2]:
#path_protocols = '/nas/public/dataset/asvspoof2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt'

In [3]:
# with open(path_protocols, 'r') as file:
#         lines = file.readlines()
#         
# file_names = [line.split()[1] for line in lines]   
# file_names[:3]

In [4]:
# def label_to_int(label):
#     return 1 if label == 'spoof' else 0

In [5]:
# # open the txt file for reading
# 
# with open(path_protocols, 'r') as f:
#     lines = f.readlines()
#     
# lines[:10]

In [6]:
# path_csv = '/nas/home/wwang/SpecResNet/data/df_eval_19.csv'
# 
# with open(path_csv, 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['', 'path', 'label'])
# 
#     for i, line in enumerate(lines):
#         # split line into columns
#         columns = line.strip().split(' ')
# 
#         # extract the relevant data
#         ordinal= i
#         file_path = (f'/nas/public/dataset/asvspoof2019/LA/ASVspoof2019_LA_eval/flac/{columns[1]}.flac')
#         label = label_to_int(columns[-1])
# 
#         # write the data on the csv
#         writer.writerow([ordinal, file_path, label])



In [7]:
# eval_19 = pd.read_csv('/nas/home/wwang/SpecResNet/data/df_eval_19.csv')
# # ordinal_list = list(eval_19[''])
# path_list = list(eval_19['path'])
# label_list = list(eval_19['label'])
# 
# # extract only basename of files
# file_names_list = [os.path.basename(file_path) for file_path in path_list]

In [8]:
# def get_missing_files(file_names_list, folder_path):
#     # Get the list of file names in the folder
#     folder_files = os.listdir(folder_path)
#     
#     missing_files = [file_name for file_name in file_names_list if file_name not in folder_files]
#     return missing_files
#     

In [9]:
# folder_path = '/nas/public/dataset/asvspoof2019/LA/ASVspoof2019_LA_eval/flac'
# get_missing_files(file_names_list, folder_path)

## Repeated spectrograms to audio

In [29]:
# get the cached spectrogram
config_path = '../config/residualnet_train_config.yaml'
config = read_yaml(config_path)
index = 0

file, label, spec = retrieve_single_cached_spec(config, index)
audio_converted, phase = spectrogram_inversion(config, index, spec, phase_info=True, phase_to_use=None)

Evaluating file /nas/public/dataset/asvspoof2019/LA/ASVspoof2019_LA_eval/flac/LA_E_2834763.flac with label 1


In [30]:
ipd.Audio(audio_converted, rate=16000)

In [32]:
len(audio_converted)

20480

In [40]:
# compute how many times it has to be repeated to reach 3s
network_input_shape = 28 * 3 #84

feature_len = spec.shape[1]  # number of time frames in the spec
X = spec

if feature_len < network_input_shape:
    num_repeats = int(network_input_shape/feature_len) + 1
    X = np.tile(X, (1, num_repeats))
    
new_spec_len = X.shape[1]
X_win = X[:, : network_input_shape]
print(f'The original spec had {feature_len} time frames which became {new_spec_len}')

The original spec had 41 time frames which became 123


In [43]:
# spectrogram that gets fed to the network
print(X_win.shape)


(1025, 84)

In [36]:
# original formula for computing the n_frames
# n_frames = 1 + ( L- n_fft)/hop_length
# inverted formula to find L:
# L = (n_frames -1 ) * hop_length + n_fft

n_frames = new_spec_len
hop_length = 512
n_fft = 2048

L = (n_frames - 1 ) * hop_length + n_fft
L

64512

In [31]:
original_audio = retrieve_single_audio(config, index)
ipd.Audio(original_audio, rate=16000)

In [33]:
len(original_audio)

22831